In [ ]:
# Clear Hugging Face model cache to avoid past errors
!rm -rf ~/.cache/huggingface

In [ ]:
#Required Libraries
!pip install transformers
!pip install torch
!pip install huggingface_hub

In [ ]:
#Upgrade the libraries
!pip install --upgrade transformers
!pip install --upgrade torch

In [ ]:
# Prompt to enter your Hugging Face token securely
import os
os.environ["HF_TOKEN"] = input("Enter your Hugging Face token: ")

In [ ]:
#Logging into the Hugging face account
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"), add_to_git_credential=True)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Variables to store order details
order = {"type": None, "size": None, "extras": None}

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=100, do_sample=True, top_k=50, top_p=0.95)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def coffee_order_chatbot():
    print("Bot: 안녕하세요! 커피 주문을 도와드리겠습니다. 어떤 종류의 커피를 원하시나요?")

    while True:
        user_input = input("You: ")

        # Exit command
        if user_input.lower() in ["exit", "quit"]:
            print("Bot: 주문을 종료하였습니다. 감사합니다!")
            break

        # Determine the stage of ordering based on the user's input and current order status
        if order["type"] is None:
            order["type"] = user_input
            response = "커피 사이즈를 선택해주세요: 작은 사이즈, 중간 사이즈, 또는 큰 사이즈."

        elif order["size"] is None:
            order["size"] = user_input
            response = "추가 옵션이 있으신가요? (예: 우유, 시럽 등)"

        elif order["extras"] is None:
            order["extras"] = user_input
            response = f"확인합니다. 주문하신 커피는 {order['type']} ({order['size']}), 추가 옵션: {order['extras']}입니다. 맞나요?"

        else:
            # Confirm and finalize order
            if "네" in user_input or "맞아요" in user_input:
                response = "주문이 완료되었습니다. 감사합니다!"
                print(f"Bot: {response}")
                break
            else:
                response = "죄송합니다. 다시 주문을 시작하겠습니다."
                order["type"], order["size"], order["extras"] = None, None, None

        print(f"Bot: {response}")

# Run the chatbot
coffee_order_chatbot()
